Of all the S tier tiles on the map, find the one with the greatest score sum of its 6 adjacent tiles.

In [ ]:
import numpy as np
from gurobipy import Model, GRB, quicksum
import pandas as pd
import json

In [60]:
def convert_complex_json_to_df(file_path):
    # Read the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Extract the tiles array which contains the data we want
    tiles_data = data['tiles']
    
    # Convert the tiles array to a DataFrame
    df = pd.DataFrame(tiles_data)
    
    # If you also want to include metadata as columns
    # you can add them like this:
    for key, value in data['metadata'].items():
        if not isinstance(value, list):  # Skip lists like tier_thresholds
            df[f'metadata_{key}'] = value
    
    return df

In [61]:
map_file_name = "civ_map_data.json"

df = convert_complex_json_to_df(map_file_name)

# print the row where x=19 and y=19
print(df[(df['x'] == 19) & (df['y'] == 19)])

      x   y               terrain         feature resource resourcetype  \
116  19  19  TERRAIN_PLAINS_HILLS  FEATURE_JUNGLE     None         None   

              continent rivers  appeal  goodyhut  normalized_score tier  \
116  CONTINENT_COLUMBIA   None    -1.0     False             174.0    A   

     metadata_min_x  metadata_max_x  metadata_min_y  metadata_max_y  
116               0              59               0              37  


In [ ]:
# create a dataframe of just S tier tiles
s_tier_df = df[df['tier'] == "S"]

# for each s tier tile, sum the scores of the 5 adjacent tiles
for index, row in s_tier_df.iterrows():
    adjacencies_sum = 0
    x_coord = row['x']
    y_coord = row['y'] 

    # assuming all of the s tier tiles have 6 adjacencies, ie they aren't on the edge of the board
    try:
        adj_1 = float(df[(df['x'] == x_coord - 1) & (df['y'] == y_coord - 1)]['normalized_score'].values)
        adj_2 = float(df[(df['x'] == x_coord - 1) & (df['y'] == y_coord)]['normalized_score'].values)
        adj_3 = float(df[(df['x'] == x_coord - 1) & (df['y'] == y_coord + 1)]['normalized_score'].values)
        adj_4 = float(df[(df['x'] == x_coord) & (df['y'] == y_coord + 1)]['normalized_score'].values)
        adj_5 = float(df[(df['x'] == x_coord + 1) & (df['y'] == y_coord)]['normalized_score'].values)
        adj_6 = float(df[(df['x'] == x_coord) & (df['y'] == y_coord - 1)]['normalized_score'].values)
        adjacencies_sum = adj_1 + adj_2 + adj_3 + adj_4 + adj_5 + adj_6


    # x/y coordinate should wrap to index 0 (or length) if the tile is on the edge of the board
    # TODO
    except Exception as e:
        # print(e)
        adjacencies_sum = -1

    s_tier_df.at[index, 'adjacencies_sum'] = adjacencies_sum

    
        

sorted_df = s_tier_df[['x', 'y', 'adjacencies_sum']].sort_values(by='adjacencies_sum', ascending=False)

# Print the sorted DataFrame with x, y, and adjacencies_sum columns
print("\nSorted coordinates with adjacencies_sum:")
print(sorted_df)


Sorted coordinates with adjacencies_sum:
     x   y  adjacencies_sum
48  50  30           1276.0
5   56  26           1092.0
24  51  30           1074.0
43   9  19           1034.0
51  10  11           1024.0
0   56  25            996.0
37  49  30            987.0
7   48  23            957.0
4   17  18            948.0
6   51  22            938.0
10  50  29            929.0
45   9  18            919.0
30  50  31            919.0
35  56  24            909.0
55  50  25            908.0
12  51  31            900.0
14  42  19            899.0
16  17  16            899.0
66  51  23            889.0
29  42  22            879.0
44  49  31            862.0
13  11  12            859.0
46  49  29            852.0
2   45  31            851.0
54  16  18            840.0
9   13  19            822.0
40  22  20            822.0
31  55  31            821.0
47  44  31            813.0
38  46  22            793.0
26  17  22            793.0
23  22  21            793.0
11   9  21            792.0
36  38

/var/folders/bp/z5zzk7d95y3bjb4hpc9nprg80000gn/T/ipykernel_98439/3281473552.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  adj_1 = float(df[(df['x'] == x_coord - 1) & (df['y'] == y_coord - 1)]['normalized_score'].values)
/var/folders/bp/z5zzk7d95y3bjb4hpc9nprg80000gn/T/ipykernel_98439/3281473552.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  adj_2 = float(df[(df['x'] == x_coord - 1) & (df['y'] == y_coord)]['normalized_score'].values)
/var/folders/bp/z5zzk7d95y3bjb4hpc9nprg80000gn/T/ipykernel_98439/3281473552.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ens